In [1]:
import cv2
import numpy as np
import face_recognition
import os
from deepface import DeepFace as dp
import time
from keras.models import load_model  

In [2]:
path = r'C:\Users\DEEPIKA\Desktop\MAJOR PROJECT\implementation\CSE-1A PHOTOS'
images = []
classNames = []
myList = os.listdir(path)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
    
encodeListKnown=np.load('encodeface.npy')

In [3]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown = findEncodings(images)

In [4]:
cap=cv2.VideoCapture(0)
#record_data={}
model = load_model('model.h5')
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

while True:
    success,img= cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)
            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Resize image to match input size of model
                gray_resized = cv2.resize(gray, (48, 48), interpolation=cv2.INTER_AREA)

                # Expand dimensions of image to match input shape of model
                gray_resized = np.expand_dims(gray_resized, axis=-1)
                gray_resized = np.expand_dims(gray_resized, axis=0)

                # Predict emotion using model
                predictions = model.predict(gray_resized)

                # Get predicted emotion label
                predicted_label = class_labels[np.argmax(predictions)]

    # Check if predicted emotion is confusion
                if predicted_label in ['Fear', 'Sad', 'Disgust']:
                    confusion = True
                else:
                    confusion = False

    # Display text above face
                if confusion:
                        cv2.putText(img, "disappoint", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                        cv2.putText(img, "not disappoint", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                        y1, x2, y2, x1 = faceLoc
                        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                        cv2.putText(img, name ,(x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    # Display video feed with emotion label
    cv2.imshow('Emotion Detection', img)      
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    time.sleep(1.2000)
cap.release()
cv2.destroyAllWindows()
print('success')

1/1 [==============================] - 0s 61ms/step
success
